In [1]:
# 03_process_all_data.ipynb
# Master Data Processing Pipeline - All Tracks, All Drivers

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pickle
import warnings
warnings.filterwarnings('ignore')

# Import our utilities - FIXED
from utils import get_lap_telemetry, detect_corners, extract_corner_features
from config import TRACKS


print("="*70)
print("TOYOTA GR CUP - MASTER DATA PROCESSING PIPELINE")
print("="*70)
print(f"\nTracks to process: {len(TRACKS)}")
print(f"Expected total races: {len(TRACKS) * 2}")
print(f"Expected drivers: ~100+ driver-race combinations\n")

# ============================================================================
# CELL 1: PROCESS ALL TRACKS - HANDLES BOTH LAP TIME FORMATS
# ============================================================================

all_corner_features = []
processing_summary = []

for track_name, config in TRACKS.items():
    track_path = config['path']
    
    if not track_path.exists():
        print(f"⚠️  {track_name}: Path not found at {track_path}, skipping")
        continue
    
    for race_idx, race_prefix in enumerate(config['races'], 1):
        try:
            if config['has_race_dirs']:
                race_dir = track_path / f"Race {race_idx}"
            else:
                race_dir = track_path
            
            if not race_dir.exists():
                print(f"⚠️  {track_name} Race {race_idx}: Directory not found")
                continue
            
            all_files = list(race_dir.glob("*.csv")) + list(race_dir.glob("*.CSV"))
            
            telemetry_files = [f for f in all_files if 'telemetry' in f.name.lower()]
            lap_time_files = [f for f in all_files if 'lap_time' in f.name.lower()]
            lap_start_files = [f for f in all_files if 'lap_start' in f.name.lower()]
            lap_end_files = [f for f in all_files if 'lap_end' in f.name.lower()]
            
            if not config['has_race_dirs']:
                telemetry_files = [f for f in telemetry_files if f"R{race_idx}" in f.name]
                lap_time_files = [f for f in lap_time_files if f"R{race_idx}" in f.name]
                lap_start_files = [f for f in lap_start_files if f"R{race_idx}" in f.name]
                lap_end_files = [f for f in lap_end_files if f"R{race_idx}" in f.name]
            
            if not telemetry_files:
                print(f"⚠️  {track_name} Race {race_idx}: No telemetry files")
                continue
            
            print(f"\n{'='*70}")
            print(f"Processing: {track_name} - Race {race_idx}")
            print(f"{'='*70}")
            
            telemetry = pd.read_csv(telemetry_files[0])
            print(f"Loaded: {len(telemetry):,} telemetry rows")
            
            # METHOD 1: Try lap_time file with 'value' column
            if lap_time_files:
                print(f"Files: {telemetry_files[0].name}, {lap_time_files[0].name}")
                lap_times = pd.read_csv(lap_time_files[0])
                
                # Check if 'value' column exists
                if 'value' in lap_times.columns:
                    print(f"    Using 'value' column from lap_time file")
                    lap_times['value'] = pd.to_numeric(lap_times['value'], errors='coerce')
                    lap_times = lap_times.dropna(subset=['value'])
                    
                    max_time_raw = lap_times['value'].max()
                    if max_time_raw > 300:
                        lap_times['lap_time_sec'] = lap_times['value'] / 1000
                    else:
                        lap_times['lap_time_sec'] = lap_times['value']
                else:
                    # METHOD 2: Calculate from lap_start and lap_end timestamps
                    print(f"    No 'value' column, trying lap_start/lap_end method...")
                    
                    if not lap_start_files or not lap_end_files:
                        print(f"    ⚠️ Missing lap_start or lap_end files, skipping")
                        continue
                    
                    lap_start = pd.read_csv(lap_start_files[0])
                    lap_end = pd.read_csv(lap_end_files[0])
                    
                    print(f"    Calculating lap times from timestamps...")
                    
                    # Convert timestamps to datetime
                    lap_start['timestamp'] = pd.to_datetime(lap_start['timestamp'])
                    lap_end['timestamp'] = pd.to_datetime(lap_end['timestamp'])
                    
                    # Merge on vehicle_id and lap
                    lap_times = pd.merge(
                        lap_start[['vehicle_id', 'lap', 'timestamp']],
                        lap_end[['vehicle_id', 'lap', 'timestamp']],
                        on=['vehicle_id', 'lap'],
                        suffixes=('_start', '_end')
                    )
                    
                    # Calculate lap time in seconds
                    lap_times['lap_time_sec'] = (lap_times['timestamp_end'] - lap_times['timestamp_start']).dt.total_seconds()
                    
                    print(f"    Calculated {len(lap_times)} lap times")
            else:
                print(f"    ⚠️ No lap time files found")
                continue
            
            # Filter reasonable lap times
            min_time, max_time = config['typical_lap_time']
            lap_times_clean = lap_times[
                (lap_times['lap_time_sec'] > min_time) & 
                (lap_times['lap_time_sec'] < max_time)
            ].copy()
            
            print(f"Clean laps: {len(lap_times_clean)} (filtered {len(lap_times) - len(lap_times_clean)} outliers)")
            
            if len(lap_times_clean) == 0:
                print(f"⚠️  No valid laps - check time range {min_time}-{max_time}s")
                actual_times = lap_times['lap_time_sec'].dropna()
                if len(actual_times) > 0:
                    print(f"    Actual range: {actual_times.min():.1f}s - {actual_times.max():.1f}s")
                continue
            
            all_drivers = lap_times_clean['vehicle_id'].unique().tolist()
            print(f"Drivers: {len(all_drivers)}")
            
            race_features = []
            successful_drivers = 0
            
            for vehicle in all_drivers:
                driver_laps = lap_times_clean[lap_times_clean['vehicle_id'] == vehicle]
                best_lap_info = driver_laps.nsmallest(1, 'lap_time_sec')
                
                if len(best_lap_info) == 0:
                    continue
                
                lap_num = best_lap_info['lap'].iloc[0]
                lap_time = best_lap_info['lap_time_sec'].iloc[0]
                
                lap_telem = get_lap_telemetry(telemetry, vehicle, lap_num)
                
                if lap_telem is None or len(lap_telem) < 100:
                    continue
                
                corners = detect_corners(lap_telem)
                
                min_corners, max_corners = config['expected_corners']
                if len(corners) < min_corners or len(corners) > max_corners:
                    continue
                
                features = extract_corner_features(lap_telem, corners)
                features['vehicle_id'] = vehicle
                features['lap'] = lap_num
                features['lap_time'] = lap_time
                features['track'] = track_name
                features['race'] = f"Race {race_idx}"
                
                race_features.append(features)
                successful_drivers += 1
                
                if successful_drivers % 5 == 0:
                    print(f"  Processed {successful_drivers}/{len(all_drivers)} drivers...")
            
            print(f"✅ Successfully processed {successful_drivers} drivers with {sum(len(f) for f in race_features)} corners")
            
            if race_features:
                race_df = pd.concat(race_features, ignore_index=True)
                all_corner_features.append(race_df)
                
                processing_summary.append({
                    'track': track_name,
                    'race': f"Race {race_idx}",
                    'drivers': successful_drivers,
                    'corners': len(race_df),
                    'avg_lap_time': race_df['lap_time'].mean()
                })
        
        except Exception as e:
            print(f"❌ Error processing {track_name} Race {race_idx}: {str(e)}")
            import traceback
            traceback.print_exc()
            continue

print(f"\n{'='*70}")
print("PROCESSING COMPLETE")
print(f"{'='*70}\n")

if all_corner_features:
    master_corners = pd.concat(all_corner_features, ignore_index=True)
    summary_df = pd.DataFrame(processing_summary)
    
    print("Dataset Summary:")
    print(summary_df.to_string(index=False))
    print(f"\n📊 Total Statistics:")
    print(f"  - Total corner features: {len(master_corners):,}")
    print(f"  - Unique drivers: {master_corners['vehicle_id'].nunique()}")
    print(f"  - Unique tracks: {master_corners['track'].nunique()}")
    print(f"  - Total races: {len(processing_summary)}")
    
    master_corners.to_csv('master_corner_features.csv', index=False)
    summary_df.to_csv('processing_summary.csv', index=False)
    print("\n✅ Saved: master_corner_features.csv")
    print("✅ Saved: processing_summary.csv")
else:
    print("❌ No data was successfully processed!")



# ============================================================================
# CELL 2: DRIVER PERFORMANCE ANALYSIS
# ============================================================================

print("\n" + "="*70)
print("DRIVER PERFORMANCE ANALYSIS")
print("="*70 + "\n")

# Calculate driver statistics per track
driver_stats = master_corners.groupby(['track', 'vehicle_id']).agg({
    'lap_time': ['mean', 'min', 'count'],
    'max_brake': 'mean',
    'apex_throttle': 'mean',
    'exit_throttle': 'mean',
    'corner_duration': 'mean'
}).reset_index()

driver_stats.columns = ['track', 'vehicle_id', 'avg_lap', 'best_lap', 'laps_analyzed', 
                        'avg_brake', 'avg_apex_throttle', 'avg_exit_throttle', 'avg_corner_duration']

# Add percentile rankings per track
for track in driver_stats['track'].unique():
    track_mask = driver_stats['track'] == track
    driver_stats.loc[track_mask, 'rank'] = driver_stats.loc[track_mask, 'best_lap'].rank()
    driver_stats.loc[track_mask, 'percentile'] = (
        100 - (driver_stats.loc[track_mask, 'rank'] / driver_stats.loc[track_mask].shape[0] * 100)
    )

driver_stats.to_csv('driver_performance_stats.csv', index=False)
print("✅ Saved: driver_performance_stats.csv")

# Show top performers per track
print("\n🏆 Top 3 Drivers per Track:")
for track in driver_stats['track'].unique():
    track_data = driver_stats[driver_stats['track'] == track].nsmallest(3, 'best_lap')
    print(f"\n{track}:")
    for idx, row in track_data.iterrows():
        print(f"  {int(row['rank'])}. {row['vehicle_id']}: {row['best_lap']:.2f}s ({row['percentile']:.0f}th percentile)")

# ============================================================================
# CELL 3: GENERATE COMPARISON DATASETS
# ============================================================================

print("\n" + "="*70)
print("GENERATING COMPARISON DATASETS")
print("="*70 + "\n")

all_comparisons = []

# For each track, compare all drivers against the fastest
for track in master_corners['track'].unique():
    track_data = master_corners[master_corners['track'] == track]
    
    # Find fastest driver
    fastest_driver = track_data.groupby('vehicle_id')['lap_time'].min().idxmin()
    fastest_lap_time = track_data[track_data['vehicle_id'] == fastest_driver]['lap_time'].min()
    
    print(f"\n{track}:")
    print(f"  Fastest: {fastest_driver} ({fastest_lap_time:.2f}s)")
    
    # Get fastest driver's corner data
    fast_corners = track_data[track_data['vehicle_id'] == fastest_driver].copy()
    
    # Compare all other drivers
    other_drivers = [d for d in track_data['vehicle_id'].unique() if d != fastest_driver]
    
    for driver in other_drivers:
        driver_corners = track_data[track_data['vehicle_id'] == driver].copy()
        driver_lap_time = driver_corners['lap_time'].iloc[0]
        
        # Compare corner by corner
        max_corners = min(len(fast_corners), len(driver_corners))
        
        for corner_num in range(1, max_corners + 1):
            fast_c = fast_corners[fast_corners['corner_num'] == corner_num]
            slow_c = driver_corners[driver_corners['corner_num'] == corner_num]
            
            if len(fast_c) == 0 or len(slow_c) == 0:
                continue
            
            fast = fast_c.iloc[0]
            slow = slow_c.iloc[0]
            
            all_comparisons.append({
                'track': track,
                'fast_driver': fastest_driver,
                'fast_lap_time': fastest_lap_time,
                'slow_driver': driver,
                'slow_lap_time': driver_lap_time,
                'time_gap': driver_lap_time - fastest_lap_time,
                'corner': corner_num,
                'brake_delta': slow['max_brake'] - fast['max_brake'],
                'apex_throttle_delta': slow['apex_throttle'] - fast['apex_throttle'],
                'duration_delta': slow['corner_duration'] - fast['corner_duration'],
                'fast_brake': fast['max_brake'],
                'slow_brake': slow['max_brake'],
                'fast_apex_throttle': fast['apex_throttle'],
                'slow_apex_throttle': slow['apex_throttle'],
                'time_lost_sec': (slow['corner_duration'] - fast['corner_duration']) * 0.04
            })
    
    print(f"  Comparisons generated: {len(other_drivers)} drivers")

comparison_df = pd.DataFrame(all_comparisons)
comparison_df.to_csv('master_comparisons.csv', index=False)
print(f"\n✅ Saved: master_comparisons.csv ({len(comparison_df):,} comparisons)")

# ============================================================================
# CELL 4: MACHINE LEARNING MODELS
# ============================================================================

print("\n" + "="*70)
print("TRAINING MACHINE LEARNING MODELS")
print("="*70 + "\n")

# Prepare ML dataset - average corner features per lap
ml_data = master_corners.groupby(['track', 'vehicle_id', 'lap', 'lap_time']).agg({
    'max_brake': 'mean',
    'brake_duration': 'mean',
    'apex_throttle': 'mean',
    'min_throttle': 'mean',
    'apex_lateral_g': 'mean',
    'exit_throttle': 'mean',
    'corner_duration': 'mean',
    'avg_steering_angle': 'mean'
}).reset_index()

print(f"ML Dataset: {len(ml_data)} samples")

# Features and target
feature_cols = ['max_brake', 'brake_duration', 'apex_throttle', 'min_throttle', 
                'apex_lateral_g', 'exit_throttle', 'corner_duration', 'avg_steering_angle']
X = ml_data[feature_cols]
y = ml_data['lap_time']

# Train Random Forest
print("\n1. Training Random Forest...")
rf_model = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42, n_jobs=-1)
rf_model.fit(X, y)
rf_score = rf_model.score(X, y)
print(f"   R² Score: {rf_score:.3f}")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n   Feature Importance:")
for idx, row in feature_importance.iterrows():
    print(f"     {row['feature']:<20s}: {row['importance']:.4f}")

feature_importance.to_csv('ml_feature_importance.csv', index=False)

# Train Gradient Boosting
print("\n2. Training Gradient Boosting...")
gb_model = GradientBoostingRegressor(n_estimators=200, max_depth=8, random_state=42)
gb_model.fit(X, y)
gb_score = gb_model.score(X, y)
print(f"   R² Score: {gb_score:.3f}")

# Save models
with open('rf_model.pkl', 'wb') as f:
    pickle.dump(rf_model, f)
with open('gb_model.pkl', 'wb') as f:
    pickle.dump(gb_model, f)

print("\n✅ Saved: rf_model.pkl")
print("✅ Saved: gb_model.pkl")
print("✅ Saved: ml_feature_importance.csv")

# ============================================================================
# CELL 5: DRIVER CLUSTERING
# ============================================================================

print("\n" + "="*70)
print("DRIVER STYLE CLUSTERING")
print("="*70 + "\n")

# Prepare clustering features
cluster_features = ml_data.groupby('vehicle_id').agg({
    'max_brake': 'mean',
    'apex_throttle': 'mean',
    'exit_throttle': 'mean',
    'corner_duration': 'mean',
    'apex_lateral_g': 'mean'
}).reset_index()

print(f"Clustering {len(cluster_features)} drivers...")

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(cluster_features[['max_brake', 'apex_throttle', 'exit_throttle', 
                                                    'corner_duration', 'apex_lateral_g']])

# K-means clustering
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
cluster_features['cluster'] = kmeans.fit_predict(X_scaled)

# Label clusters based on characteristics
cluster_labels = {
    0: "Smooth & Consistent",
    1: "Aggressive Late-Braker",
    2: "Early Apex Specialist",
    3: "High-Speed Demon"
}

# Analyze clusters
print("\nCluster Characteristics:")
for cluster_id in range(4):
    cluster_data = cluster_features[cluster_features['cluster'] == cluster_id]
    print(f"\n{cluster_labels.get(cluster_id, f'Cluster {cluster_id}')} ({len(cluster_data)} drivers):")
    print(f"  Avg Brake: {cluster_data['max_brake'].mean():.1f}")
    print(f"  Avg Apex Throttle: {cluster_data['apex_throttle'].mean():.1f}%")
    print(f"  Avg Exit Throttle: {cluster_data['exit_throttle'].mean():.1f}%")
    print(f"  Example drivers: {', '.join(cluster_data['vehicle_id'].head(3).tolist())}")

cluster_features['cluster_label'] = cluster_features['cluster'].map(cluster_labels)
cluster_features.to_csv('driver_clusters.csv', index=False)

print("\n✅ Saved: driver_clusters.csv")

# ============================================================================
# CELL 6: EXPORT SUMMARY
# ============================================================================

print("\n" + "="*70)
print("EXPORT SUMMARY")
print("="*70 + "\n")

print("📁 Files Generated:")
print("  1. master_corner_features.csv - All corner data from all tracks")
print("  2. processing_summary.csv - Track/race processing stats")
print("  3. driver_performance_stats.csv - Driver rankings and stats")
print("  4. master_comparisons.csv - All driver comparisons")
print("  5. ml_feature_importance.csv - ML insights")
print("  6. driver_clusters.csv - Driver style classifications")
print("  7. rf_model.pkl - Random Forest model")
print("  8. gb_model.pkl - Gradient Boosting model")

print("\n📊 Dataset Statistics:")
print(f"  Total Corners Analyzed: {len(master_corners):,}")
print(f"  Unique Drivers: {master_corners['vehicle_id'].nunique()}")
print(f"  Tracks: {master_corners['track'].nunique()}")
print(f"  Total Comparisons: {len(comparison_df):,}")
print(f"  ML Training Samples: {len(ml_data)}")

print("\n✅ ALL DATA PROCESSING COMPLETE!")
print("\nNext steps:")
print("  1. Update app.py with multi-track support")
print("  2. Test dashboard with new data")
print("  3. Add advanced visualizations")


TOYOTA GR CUP - MASTER DATA PROCESSING PIPELINE

Tracks to process: 7
Expected total races: 14
Expected drivers: ~100+ driver-race combinations


Processing: Sebring - Race 1
Loaded: 14,216,998 telemetry rows
Files: sebring_telemetry_R1.csv, sebring_lap_time_R1.csv
    Using 'value' column from lap_time file
Clean laps: 352 (filtered 109 outliers)
Drivers: 20
  Processed 5/20 drivers...
  Processed 10/20 drivers...
✅ Successfully processed 14 drivers with 235 corners

Processing: Sebring - Race 2
Loaded: 2,394,340 telemetry rows
Files: sebring_telemetry_R2.csv, sebring_lap_time_R2.csv
    Using 'value' column from lap_time file
Clean laps: 277 (filtered 150 outliers)
Drivers: 19
✅ Successfully processed 0 drivers with 0 corners

Processing: Sonoma - Race 1
Loaded: 27,476,808 telemetry rows
Files: sonoma_telemetry_R1.csv, sonoma_lap_time_R1.csv
    Using 'value' column from lap_time file
Clean laps: 260 (filtered 786 outliers)
Drivers: 31
  Processed 5/31 drivers...
  Processed 10/31 dr

In [5]:
# DIAGNOSTIC CELL - Check Barber telemetry parameters
print("Checking Barber telemetry parameters...")
barber_telem = pd.read_csv('sebring/Sebring/Race 2/sebring_telemetry_R2.csv', nrows=10000)
print(f"Sebring columns: {barber_telem.columns.tolist()}")
print(f"\nUnique telemetry_name values:")
# print(barber_telem['telemetry_name'].unique()[:20])  # Show first 20


Checking Barber telemetry parameters...
Sebring columns: ['meta_source', 'meta_time', 'meta_event', 'meta_session', 'timestamp', 'vehicle_id', 'outing', 'lap', 'value', 'expire_at']

Unique telemetry_name values:
